In [35]:
import numpy as np
import torch
import matplotlib.pyplot as plt
from collections import Counter
import itertools
import torch.nn.functional as f

In [11]:
with open(r"names.txt",'r') as f:
    words = [i.strip('\n') for i in f.readlines()]

Building a TriGram Language Model

In [12]:
#get all characters 
chars = sorted(list(set(''.join(words))))
bigram_starts = [''.join([i,j]) for i,j in list(itertools.permutations(chars,2))]
bigram_with_dot = [''.join(['.',i]) for i in chars]
bigram_with_itself = [''.join([i,i]) for i in chars]
final_bigram_list = bigram_with_dot + bigram_starts + bigram_with_itself
final_bigram_list = sorted(final_bigram_list)

In [13]:
# bigram_starts.extend(bigram_with_itself)
# bigram_starts = sorted(bigram_starts)
bsti = {j:i for i,j in enumerate(final_bigram_list)}
itbs = {i:j for i,j in enumerate(final_bigram_list)}

Main counts tensor will be of 702 x 27 dimensions

In [14]:
chars.extend(['.'])
itoc = {i:j for i,j in enumerate(chars)}
ctoi = {j:i for i,j in enumerate(chars)}

In [15]:
#Main counts tensor
N = torch.zeros(size = (len(final_bigram_list),27),dtype = torch.int32)

In [16]:
for word in words:
    chs = '.' + word + '.'
    for i,j in enumerate(chs):
        condition = chs[i:i+2]
        result = chs[i+2]
        idx_row = bsti[condition]
        idx_col = ctoi[result]
        N[idx_row,idx_col] += 1
        if result == '.':
            break

In [17]:
#add fake counts for smoothing
P = N + 1
#normalizing P
log_likelihood = 0.0
n = 0
P = P/P.sum(1,keepdims = True)
for word in words:
    chs = '.' + word + '.'
    for i,j in enumerate(chs):
        #e.g: .emma.
        #P(m|.e), P(m|em), P(a|mm),P(.|ma)
        condition = chs[i:i+2]
        result = chs[i + 2]
        idx_row = bsti[condition]
        idx_col = ctoi[result]
        prob = P[idx_row,idx_col]
        logprob = torch.log(prob)
        log_likelihood += logprob.item()
        n += 1
        if result == '.':
            break
print(f'Negative log likelihood: {-log_likelihood/n: .4f}')


Negative log likelihood:  2.0931


Neural Net Model

Create tensor for one hot encoding. 

The one hot vector will be of length 702.

In [58]:
xs = []
ys = []
for word in words[:50]:
    chs = '.' + word + '.'
    for i in range(len(chs)):
        input = chs[i:i+2]
        result = chs[i + 2]
        idx_input = bsti[input]
        idx_output = ctoi[result]
        xs.append(idx_input)
        ys.append(idx_output)
        if result == '.':
            break

In [59]:
lr = 50
iter = 50
xenc = f.one_hot(torch.tensor(xs).long(),num_classes = len(bsti)).float()
g = torch.Generator().manual_seed(2147483647)
W = torch.randn((xenc.shape[1],len(ctoi)),generator=g,requires_grad=True).float()

for i in range(iter):
    logits = xenc @ W
    counts = logits.exp()
    probs = counts/counts.sum(1,keepdims = True)

    #loss
    class_probs = probs[torch.arange(len(ys)),ys]
    loss = -1 * class_probs.log().mean()
    print(f"Loss : {loss.item(): .4f}")

    W.grad = None
    loss.backward()

    #update
    W.data += - lr*W.grad


Loss :  3.6238
Loss :  3.3120
Loss :  3.0412
Loss :  2.8108
Loss :  2.6103
Loss :  2.4321
Loss :  2.2728
Loss :  2.1306
Loss :  2.0038
Loss :  1.8906
Loss :  1.7894
Loss :  1.6987
Loss :  1.6172
Loss :  1.5438
Loss :  1.4774
Loss :  1.4172
Loss :  1.3626
Loss :  1.3129
Loss :  1.2677
Loss :  1.2266
Loss :  1.1891
Loss :  1.1550
Loss :  1.1239
Loss :  1.0956
Loss :  1.0699
Loss :  1.0465
Loss :  1.0251
Loss :  1.0057
Loss :  0.9880
Loss :  0.9718
Loss :  0.9570
Loss :  0.9435
Loss :  0.9311
Loss :  0.9198
Loss :  0.9093
Loss :  0.8997
Loss :  0.8908
Loss :  0.8825
Loss :  0.8749
Loss :  0.8678
Loss :  0.8612
Loss :  0.8550
Loss :  0.8493
Loss :  0.8439
Loss :  0.8388
Loss :  0.8341
Loss :  0.8296
Loss :  0.8254
Loss :  0.8214
Loss :  0.8177


In [60]:
rrrrrrrr

NameError: name 'rrrrrrrr' is not defined

In [ ]:
W.shape

torch.Size([702, 287])

In [ ]:
(xenc @ W).shape

torch.Size([287, 27])

In [ ]:
xenc.shape

torch.Size([196113])

In [ ]:
rrrrr

NameError: name 'rrrrr' is not defined

In [ ]:
start_tensor = torch.zeros(size = (1,len(bigram_starts)),dtype = torch.int32)
for word in words:
    #take the first two characters
    char_two = word[:2]
    idx = bsti[char_two]
    start_tensor[0,idx] += 1